# Imports

In [4]:
import pandas as pd

import numpy as np

# import matplotlib.pyplot as plt

# import seaborn as sns

import tensorflow as tf

import re

import nltk

from nltk.corpus import stopwords

from nltk.stem import WordNetLemmatizer

from nltk.tokenize import word_tokenize,sent_tokenize

from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, BatchNormalization, MaxPooling2D, Embedding, LSTM, Bidirectional

from tensorflow.keras.models import Sequential

from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.preprocessing.sequence import pad_sequences


# Data

In [5]:
!kaggle datasets download -d yasserh/imdb-movie-ratings-sentiment-analysis

'kaggle' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
!unzip imdb-movie-ratings-sentiment-analysis.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
df=pd.read_csv('movie.csv')

df.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


### Here that is a data classification so i dont need the sequence

### so what i will do after cleaning:

### 1- Tokenize and remove punctuation

### 2- Remove stop words

### 3- lemmatization

### 4- do TF-IDF

### 5- Modeling

### cleaning data

In [8]:
df.isna().sum()

text     0
label    0
dtype: int64

In [9]:
df.duplicated().sum()

277

In [10]:
df.drop_duplicates(inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39723 entries, 0 to 39999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    39723 non-null  object
 1   label   39723 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 931.0+ KB


In [12]:
df.columns

Index(['text', 'label'], dtype='object')

In [13]:
df['label'].value_counts() ### to check data imbalance

label
1    19908
0    19815
Name: count, dtype: int64

In [14]:
df['label'].unique()

array([0, 1], dtype=int64)

In [15]:
nltk.download('punkt')

nltk.download('stopwords')

nltk.download('wordnet')
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [16]:
stop_words = set(stopwords.words('english'))

lemmatizer = WordNetLemmatizer()



def clean_text(text):

    if isinstance(text, list):

        text = " ".join(text)

    text = text.lower()

    text = re.sub(r"[^a-z!?',.\s ]", "", text)

    text = re.sub(r'\b(`|``)\b', '"', text)

    text = re.sub(r"\s+", " ", text)

    text = re.sub(r"\b(im)\b","i'm",text)

    text = re.sub(r'\b(br|b)\b', '', text)

    text = re.sub(r"\b(\s's)/b","'s",text)

    tokens = word_tokenize(text)

    tokens = [token for token in tokens if token not in stop_words]

    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    cleaned_text = " ".join(tokens)

    return cleaned_text


In [17]:
for index, row in df.iterrows():

    df.loc[index, 'text'] = clean_text(row['text'])

KeyboardInterrupt: 

In [15]:
df['text'][39995]

"western union something forgotten classic western ! perhaps reason lie fact unavailability dvd united state . however , lost appeared region england . blessing way incongruous totally ironic one considers movie depicting founding establishment uniquely american organization western union telegraph company without region release . beggar belief ! simply n't make sense ! produced fox western union directed fritz lang . second occasion great german director undertook direct western ! done excellent job year fox 's return frank james would one western outing splendid rancho notorious . lang ford hawk western union turned fine solid western hold well . beautifully photographed early three strip technicolor edward cronjager boasted good cast headed robert young , randolph scott dean jagger . female lead taken virginia gilmore really little picture . actress never made anything career . presence merely cosmetic . curious robert young top billing scott ! clearly scott 's picture beginning fir

# train-test-split

In [16]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(df['text'],df['label'],test_size=0.2,random_state=42)

# Prepare data for Embedding layer

### 1-Tokenizer

### 2-sequences

### 3-maxsequence

### 4-padding

### 5-model

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.preprocessing.sequence import pad_sequences

In [18]:
from collections import Counter
tokenzier=Tokenizer()

tokenzier.fit_on_texts(df['text'])

length=len(tokenzier.word_index) + 1

seq_train=tokenzier.texts_to_sequences(df['text'])

lengths = [len(i) for i in seq_train]

length_counts = Counter(lengths)

most_common_length = length_counts.most_common(1)[0][0]

padded_sequences = pad_sequences(seq_train, maxlen=most_common_length, padding='post')

df['padded_text'] = list(padded_sequences)


In [139]:
x = np.array(padded_sequences)

y = np.array(df['label'])

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [140]:
from tensorflow.keras.regularizers import l2 
model = Sequential()

model.add(Embedding(input_dim=length, output_dim=100))

model.add(LSTM(4,kernel_regularizer=l2(0.001)))

model.add(Dropout(0.7))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [141]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=128,callbacks=[early_stopping])

Epoch 1/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.5678 - loss: 0.6969 - val_accuracy: 0.8155 - val_loss: 0.4709
Epoch 2/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.7677 - loss: 0.4812 - val_accuracy: 0.8373 - val_loss: 0.4190
Epoch 3/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8183 - loss: 0.4092 - val_accuracy: 0.8439 - val_loss: 0.3958
Epoch 4/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8322 - loss: 0.3648 - val_accuracy: 0.8409 - val_loss: 0.4053
Epoch 5/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8346 - loss: 0.3559 - val_accuracy: 0.8423 - val_loss: 0.3988
Epoch 6/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8506 - loss: 0.3118 - val_accuracy: 0.8209 - val_loss: 0.4766
Epoch 7/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8515 - loss: 0.3148 - val_accuracy: 0.8396 - val_loss: 0.4120
Epoch 8/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8586 - loss: 0.2889 - val_accuracy: 

In [142]:
loss, accuracy = model.evaluate(X_test, y_test)

print(f'Validation Accuracy: {accuracy:.2f} and {loss}')

373/373 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8445 - loss: 0.3963
Validation Accuracy: 0.84 and 0.3978286683559418


In [144]:
def predict(new_text):
    cleaned_new_text = clean_text(new_text)
    new_sequence = tokenzier.texts_to_sequences([cleaned_new_text])
    new_padded_sequence = pad_sequences(new_sequence, maxlen=most_common_length,padding='post')
    prediction = model.predict(new_padded_sequence)
    if prediction[0][0]>0.5:
        return 'Good Review'
    else:
        return 'Bad Review'


In [145]:
predict('the film is too good')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step


'Good Review'

In [146]:
predict('the film is too bad')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


'Bad Review'

In [147]:
predict('Directing is very poor')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


'Bad Review'

# Saving The Model

In [148]:
import pickle

# Save the tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenzier, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [149]:
from keras.models import load_model

# Save the model
model.save('movie_review_model.h5')
